In [29]:
from opentrons import robot, containers, instruments
      
robot.reset()

# your file should be a .csv file with the rows (0 to 11, 12 total) of each 96 well plate that contain sample. 
# Each plate should be on a new line (starting from plate 1, then 2, 3, 4).
# Each line contains the row numbers in that plate that have sample followed by a colon and the dilution factor
# for that row. Separate each row number:dilution factor with a comma and reference each row using the 
# zero-indexed number for the row (between 0 and 11).
# If there are no rows with sample in a certain plate, simply move on to the next plate leaving that line blank.

# Example (do not include the # marks):

# 0:5, 2:10, 3:7, 7:5, 11:5
#
# 0:1, 2:2, 3:3, 11:12
# 4:7, 5:2

# This means: 
# plate 1 row 0 dilution factor 5, row 2 factor 10, row 3 factor 7, 7:5, 12:5
# no sample in plate 2
# plate 3 row 0 factor 1, row 2 factor 2, row 3 factor 3, row 12 factor 12
# plate 4 row 4 factor 7, row 5 factor 2 

# This opens the file with absolute path (will be different depending on operating system)
# file paths on Windows look more like 'C:\\path\\to\\your\\csv_file.csv'
# on mac more like '/path/to/your/csv_file.csv'
path = 'C:\\Users\\calel_000\\Libraries\\Documents\\Opentrons\\Protocols\\UTennesseeExample.csv'

#96 well source plate
plate1 = containers.load('96-deep-well', 'A1')
plate2 = containers.load('96-deep-well', 'B1')
plate3 = containers.load('96-deep-well', 'C1')
plate4 = containers.load('96-deep-well', 'D1')

plates = [plate1, plate2, plate3, plate4]

#96 well destination plate
dest_plate = containers.load('384-plate', 'E1')

dict_source = [row for row in plate1.rows()] + [row for row in plate2.rows()] + [row for row in plate3.rows()] + [row for row in plate4.rows()]

dict_dest = [row.wells('A', length=8, step=2) for row in dest_plate.rows()] + [row.wells('B', length=8, step=2) for row in dest_plate.rows()]

platemap = dict(zip(dict_source, dict_dest))
platemap2 = dict(zip(dict_dest, dict_source))

# Step 1: Have user select the wells that contain samples in the 96 well plate
# Step 2: Specify the dilution factor for row (e.g. 10x)
# Note : User should design plate so that each column of the deep well plate has the same dilution factor and thus the multichannel pipette can be used.
with open(path) as my_file:

    # save all well source locations from CSV file into a list
    sources = []
    # save all well destination locations from CSV file into a list
    destinations = []
    # save all dilution factors from CSV file into a list
    dilutions = []

    # loop through each line (the rows and dilution factor pairs for each plate)
    for platenum, line in enumerate(my_file.read().splitlines()):
        # separate by the comma to find different row:dilutions
        if line:
            rows = line.split(',')
            source = []
            dfactors = []
            for row in rows:
                source.append(row.split(':')[0].strip()) #get the source from pair
                dfactors.append(row.split(':')[1].strip()) #get the dilution from pair
            plate = plates[platenum]
            for row in source:
                sources.append(plate.rows(int(row))) # save the source
                destinations.append(platemap.get(plate.rows(int(row)))) # save the destination
            dilutions += [int(factor) for factor in dfactors]
            

#tip rack for p200 and p50 pipette
tip200_rack = containers.load('tiprack-200ul', 'A2')

#trash to dispose of tips
trash = containers.load('trash-box', 'D2')

#p200 (20 - 200 uL) (single)
p200single = instruments.Pipette(
    axis='b',
    name='p200single',
    max_volume=200,
    min_volume=20,
    channels=1,
    trash_container=trash,
    tip_racks=[tip200_rack])

#p50 (5 - 50 uL) (multi)
p50single = instruments.Pipette(
    axis='b',
    name='p50single',
    max_volume=50,
    min_volume=5,
    channels=8,
    trash_container=trash,
    tip_racks=[tip200_rack2])

# Step 3: Add dilution buffer (1x PBS) based on the dilution factor (e.g. for a dilution factor of 5x add 40 uL of dilution buffer with 10 uL of sample)
# Note 1: see attached excel file for how 384 well plates correspond to 96 well plates
# Note 2: The max volume of a well in the 384 well plate is about 90 ul so the dilution buffer + the sample cannot exceed this value. The minimum volume should also be 30 uL, so for a 1x dilution factor there should be a minimum of 30 uL of sample taken).
# Step 3: For each well in the deep well plate, transfer sample into the corresponding well of the 384 well plate according to the rules in attached excel sheet.
# Step 4: Pause to wait for user to move 384 well plate to take Microplate reader measurement.
# Step 5: Transfer 20 uL from 384 well plate to the 96 well plate (according to corresponding well rules).
# Step 6: Fill each well with a sample in the 96 well plate with 180 uL of flow cytometry reagent
# Step 7: Mix flow cytometry reagent thoroughly by pipetting and stirring